In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models, datasets
import pytorch_lightning as pl

In [2]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from PIL import Image
%matplotlib inline
import cv2
from glob import glob
import numpy as np
import pandas as pd

In [3]:
pl.seed_everything(42)

Global seed set to 42


42

In [4]:
train_data_dir = './data/training_set/training_set'
test_data_dir = './data/test_set/test_set'

In [5]:
class CatsDogs(Dataset):
    def __init__(self, folder):
        cats = glob(folder+'/cats/*.jpg')
        dogs = glob(folder+'/dogs/*.jpg')
        self.fpaths = cats[:500] + dogs[0:500]
        self.normalize = transforms.Normalize(
                            mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]
                        )
        self.targets = [fpath.split('/')[-1].startswith('dog') for fpath in self.fpaths]
    
    def __len__(self): return len(self.fpaths)

    def __getitem__(self, ix):
        f = self.fpaths[ix]
        target = self.targets[ix]
        im = (cv2.imread(f)[:, :, ::-1])
        im = cv2.resize(im, (244, 244))
        im = torch.tensor(im/255)
        im = im.permute(2, 0, 1)
        im= self.normalize(im)
        return im.float(), torch.tensor([target]).float()

In [6]:
class CatsVDogs(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = models.vgg16(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.model.classifier = nn.Sequential(
                            nn.Flatten(),
                            nn.Linear(512, 128),
                            nn.ReLU(),
                            nn.Dropout(0.2),
                            nn.Linear(128, 1),
                            nn.Sigmoid()
                        )
        self.metric = nn.BCELoss()
        
    def forward(self, x):
        x = self.model(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def train_dataloader(self):
        train = CatsDogs(train_data_dir)
        return DataLoader(train, batch_size=32, shuffle=True, drop_last=True)
    
    def test_dataloader(self):
        val = CatsDogs(test_data_dir)
        return DataLoader(val, batch_size=32, shuffle=True, drop_last=True)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.metric(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.metric(y_hat, y)
        self.log('test_loss', loss)
        return loss

In [7]:
model = CatsVDogs()
trainer = pl.Trainer(max_epochs=5)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name   | Type    | Params
-----------------------------------
0 | model  | VGG     | 14.8 M
1 | metric | BCELoss | 0     
-----------------------------------
65.8 K    Trainable params
14.7 M    Non-trainable params
14.8 M    Total params
59.122    Total estimated model params size (MB)


Epoch 4: 100%|████████████████████████████████████████████████████| 31/31 [15:30<00:00, 30.03s/it, loss=0.0265, v_num=2]
